In [1]:
import numpy as np
import pandas as pd
import re

from collections import defaultdict

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import kendalltau, spearmanr

In [2]:
data_players = pd.read_pickle("./chgk/players.pkl")
data_players = pd.DataFrame.from_dict(data_players, orient='index')

In [3]:
data_players.head()

,id,name,patronymic,surname
1,1,Алексей,None,Абабилов
10,10,Игорь,,Абалов
11,11,Наталья,Юрьевна,Абалымова
12,12,Артур,Евгеньевич,Абальян
13,13,Эрик,Евгеньевич,Абальян


1 Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl).

In [4]:
data_tournaments = pd.read_pickle('./chgk/tournaments.pkl')
data_tournaments = pd.DataFrame.from_dict(data_tournaments, orient='index')

data_tournaments.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
1,1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


Для унификации предлагаю: взять в тренировочный набор турниры с dateStart из 2019 года; в тестовый — турниры с dateStart из 2020 года.

In [5]:
train_data_tournaments = data_tournaments[[date[:4]=='2019' for date in data_tournaments['dateStart']]]
test_data_tournaments = data_tournaments[[date[:4]=='2020' for date in data_tournaments['dateStart']]]

In [6]:
data_results = pd.read_pickle('./chgk/results.pkl')

2 Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков.

Замечания и подсказки:
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [7]:
def get_res_info(data_results):
    data_tour = {}

    for tour_id, tour_teams in data_results.items():
        cur_tour = defaultdict(dict)
        for team in tour_teams:

            if ('mask' not in team or
                not team['mask'] or
                not team['teamMembers'] or
                not bool(re.match('^[10]+$', team['mask']))):
                continue

            team_id = team['team']['id']
            cur_tour[team_id]['mask'] = team['mask']
            cur_tour[team_id]['players'] = [player['player']['id'] for player in team['teamMembers']]

        mask_size = [len(team['mask']) for team in cur_tour.values()]
        if mask_size and mask_size.count(mask_size[0]) == len(mask_size):
            data_tour[tour_id] = cur_tour
    return data_tour

In [8]:
def get_df_question(data_tournaments, data_tour_info):
    data_question = []
    cur_quest = 0
    for tour_id in data_tournaments['id']:
        if tour_id not in data_tour_info:
            continue

        for team_id, team_info in data_tour_info[tour_id].items():
            quest_nums = np.arange(cur_quest, cur_quest + len(team_info['mask']))
            
            for player_id in team_info['players']:
                for quest_id, ans in enumerate(team_info['mask']):
                    data_question.append({'tour_id': tour_id,
                                         'team_id':team_id,
                                         'player_id':player_id,
                                         'question_id': quest_nums[quest_id],
                                         'is_answered': ans})
        cur_quest += len(team_info['mask'])
    return pd.DataFrame.from_dict(data_question)

In [9]:
data_tour = get_res_info(data_results)

In [10]:
train_data_question = get_df_question(train_data_tournaments, data_tour)
train_data_question.head()

,tour_id,team_id,player_id,question_id,is_answered
0,4772,45556,6212,0,1
1,4772,45556,6212,1,1
2,4772,45556,6212,2,1
3,4772,45556,6212,3,1
4,4772,45556,6212,4,1


In [11]:
encoder = OneHotEncoder(handle_unknown='ignore')
train_features = encoder.fit_transform(train_data_question[['player_id', 'question_id']])

In [12]:
baseline_logreg = LogisticRegression(solver='liblinear', random_state=42)
baseline_logreg.fit(train_features, train_data_question['is_answered'])

LogisticRegression(random_state=42, solver='liblinear')

In [13]:
def get_player_rating(rating, player_names, data_question):
    players_rating = pd.DataFrame((player_names['surname'] + ' ' 
                                  + player_names['name'] + ' ' 
                                  + player_names['patronymic']).dropna().items(), 
                                 columns=['player_id', 'fio'])
    
    train_uniq_player = data_question['player_id'].unique()
    players_rating = players_rating[[player in train_uniq_player for player in players_rating['player_id']]]
    players_rating['rating'] = rating[:len(players_rating)]
    return players_rating.sort_values(by='rating', ascending=False).reset_index(drop=True)

In [14]:
players_rating_train = get_player_rating(baseline_logreg.coef_[0], data_players, train_data_question)
players_rating_train

,player_id,fio,rating
0,27403,Руссо Максим Михайлович,4.069689
1,4270,Брутер Александра Владимировна,3.932983
2,28751,Семушин Иван Николаевич,3.883775
3,27822,Савченков Михаил Владимирович,3.855190
4,30270,Спешков Сергей Леонидович,3.758045
...,...,...,...
55146,209400,Шапуров Илья Владиславович,-3.732053
55147,209401,Казарин Михаил Максимович,-3.732053
55148,203842,Балакина Вероника Андреевна,-3.759571
55149,203845,Бурштын Андрей Кириллович,-3.878246


3 Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

In [15]:
def get_test_data_question(test_data_tournaments, data_tour):
    test_data_question = get_df_question(test_data_tournaments, data_tour)
    test_data_question[["is_answered"]] = test_data_question[["is_answered"]].apply(pd.to_numeric)
    test_data_question['is_answered'] = test_data_question.groupby(['tour_id', 'team_id', 
                                                                    'player_id'])['is_answered'].transform('sum')
    test_data_question['question_id'] = -1
    test_data_question = test_data_question.drop_duplicates()
    return test_data_question

In [16]:
test_data_question = get_test_data_question(test_data_tournaments, data_tour)
test_data_question.head()

,tour_id,team_id,player_id,question_id,is_answered
0,5414,66120,18490,-1,33
36,5414,66120,116901,-1,33
72,5414,66120,8532,-1,33
108,5414,66120,42346,-1,33
144,5414,66120,123190,-1,33


In [17]:
test_features = encoder.transform(test_data_question[['player_id', 'question_id']])
predict = baseline_logreg.predict_proba(test_features)[:,1]

In [18]:
def get_correlation_values(data, predict):
    data['predict'] = predict.copy()
    #вероятность того, что хотя бы 1 из команды ответил
    data['prob_ans'] = data.groupby(['tour_id', 'team_id'])['predict'].transform(lambda x: 1 - np.prod(1 - x))

    data = data[['tour_id', 'team_id', 'is_answered', 'prob_ans']].drop_duplicates().reset_index(drop=True)

    #real rating 
    data = data.sort_values(['tour_id', 'is_answered'], ascending=False).reset_index(drop=True)
    data['real_rating'] = data.groupby(['tour_id'])['is_answered'].transform(lambda x: range(1, len(x) + 1))

    #pred rating 
    data = data.sort_values(['tour_id', 'prob_ans'], ascending=False).reset_index(drop=True)
    data['pred_rating'] = data.groupby(['tour_id'])['prob_ans'].transform(lambda x: range(1, len(x) + 1))

    spearmanr_corr_value = data.groupby(['tour_id']).apply(lambda x: spearmanr(x['real_rating'], x['pred_rating']).statistic).mean()
    kendalltau_corr_value = data.groupby(['tour_id']).apply(lambda x: kendalltau(x['real_rating'], x['pred_rating']).statistic).mean()
    
    return spearmanr_corr_value, kendalltau_corr_value

In [19]:
spearmanr_corr_value, kendalltau_corr_value = get_correlation_values(test_data_question, predict)
print(f'spearamanr value = {spearmanr_corr_value}; kendalltau value = {kendalltau_corr_value}')

spearamanr value = 0.7803548003150785; kendalltau value = 0.6116458432819316


4 Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


В прошлом пункте мы считали, если команда ответила, то и каждый участник ответил. Если команда не ответила, то никто из участников не овтетил

- скрытая переменная - как ответил участник в рамках одной команды
- наблюдаемая переменная - как ответила команда 

- - На E-шаге (expectation) изменяем скрытые переменные
- - На M-шаге (maximization) обучаем логистическую регрессию на полученных скрытых переменных

In [20]:
import torch
from torch import nn

In [21]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [22]:
def convert_cst2tensor(data):
    coo = data.tocoo()
    i = torch.LongTensor(np.vstack((coo.row, coo.col))).to(DEVICE)
    v = torch.FloatTensor(coo.data).to(DEVICE)
    return torch.sparse.FloatTensor(i, v, torch.Size(coo.shape)).to(DEVICE)

In [44]:
class LogReg:
    
    class _LogReg(nn.Module):
        def __init__(self, in_channel):
            super().__init__()
            self.linear = nn.Linear(in_channel, 1)

        def forward(self, x):
            return torch.sigmoid(self.linear(x))
    
    
    def __init__(self, base_coef, base_bias):
        self.logreg_model = self._LogReg(base_coef.shape[1]).to(DEVICE)
        self.logreg_model.linear.weight = nn.parameter.Parameter(torch.as_tensor(base_coef, dtype=torch.float32))
        self.logreg_model.linear.bias = nn.parameter.Parameter(torch.as_tensor(base_bias, dtype=torch.float32))
        self.loss = nn.BCELoss()
        self.optim = torch.optim.SGD(self.logreg_model.parameters(), momentum=0.75, lr=0.1)
    
    
    def fit(self, features, target, num_epoch=20):
        features = convert_cst2tensor(features)
        target = torch.as_tensor(target).to(DEVICE).unsqueeze(1)
        self.logreg_model.train()

        for _ in range(num_epoch):
            self.logreg_model.zero_grad()
            pred = self.logreg_model(features)
            loss = self.loss(pred, target)
            loss.backward()
            self.optim.step()
        return 
    
    
    def predict(self, features):
        features = convert_cst2tensor(features)
        self.logreg_model.to(DEVICE)
        with torch.no_grad():
            return self.logreg_model(features).detach().cpu().numpy()
        
    def get_coef(self):
        return self.logreg_model.linear.weight.detach().cpu().numpy()[0]

In [52]:
def em_algo(base_model, tr_data, tr_features, test_data, test_features, num_iter=5):
    
    data = tr_data.copy()
    model = LogReg(base_model.coef_, base_model.intercept_[0])
    for _ in range(num_iter):
        train_pred = model.predict(tr_features).squeeze()
        
        #make expectation iter
        data['pred'] = 1 - train_pred.copy()
        player_skill = data.groupby(['team_id', 'question_id']).agg({'pred': 'prod'}).reset_index()
        player_skill['pred'] = 1 - player_skill['pred']
        player_skill = data[['team_id', 'question_id']].merge(player_skill)
        
        # заметно медленнее работает 
#         data['player_skill'] = data.groupby(['team_id', 'question_id'])['pred'].transform(lambda x: 1 - np.prod(1 - x))

        data['pred'] = np.clip(data['pred']/player_skill['pred'], 0, 1).values
        data.loc[data['is_answered']==0,'pred'] = 0
        
        #make maximization iter
        model.fit(tr_features, data['pred'])
        
        test_pred = model.predict(test_features)
        spearmanr_corr_value, kendalltau_corr_value = get_correlation_values(test_data, test_pred)
        print(f'spearamanr value = {spearmanr_corr_value}; kendalltau value = {kendalltau_corr_value}\n')
    return model

In [53]:
em_model = em_algo(baseline_logreg, train_data_question, train_features, test_data_question, test_features)

spearamanr value = 0.7669381416020993; kendalltau value = 0.5992687758507924

spearamanr value = 0.7750432673543266; kendalltau value = 0.6065167364370256

spearamanr value = 0.7705292392410755; kendalltau value = 0.6025995644372697

spearamanr value = 0.7732606199679813; kendalltau value = 0.6050979386881983

spearamanr value = 0.7716155713108259; kendalltau value = 0.6037176555994278



*Заметим что целевые метрики растут, но не так значительно*

5. А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).
